# AMI

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
episode_number = 3
# episode_number = int(episode_number)
episode = 'episode' + '{:03d}'.format(episode_number)
# Set the directory for the episode
directory = 'podcast/' + episode 

In [4]:
import os
articles = os.listdir(f'podcast/{episode}/text')
articles

['Les embauches de cadres marquent le pas mais résistent toujours.txt',
 'Nikki Haley joue sa carte dans la primaire républicaine.txt',
 'Nucléaire - les petits réacteurs essuient un revers aux Etats-Unis.txt',
 'Présidentielle américaine 2024 - Donald Trump fait la course en tête.txt',
 'Trump II - le même en pire.txt',
 '« Je suis devenu président grâce à ma marque », se vante Trump à son procès.txt']

In [5]:
article = articles[0]
article

'Les embauches de cadres marquent le pas mais résistent toujours.txt'

## OpenAI summary

In [6]:
os.mkdir(f'podcast/{episode}/summary')

In [7]:
# Read the text from the file
with open(f'podcast/{episode}/text/' + article, 'r',encoding='utf-8') as f:
    text = f.read()

In [8]:
import openai, os
openai.api_key = os.getenv("OPENAI_API_KEY")
def summarize(text):
    inst = '''Resume l'article suivant en moins de 1000 caracteres:''' 
    completion = openai.chat.completions.create(
        model='gpt-3.5-turbo-1106',
        messages= [
            {'role': 'system', 'content': inst },
            {'role': 'user', 'content': text }]
    )
    return completion.choices[0].message.content

In [9]:
summary = summarize(text)

In [10]:
len(summary)

565

In [11]:
summary = article.replace('.txt', '\n')+summary

In [12]:
# Write the summary string to the file
with open(f'podcast/{episode}/summary/' + article, 'w', encoding='utf-8') as f:
    f.write(summary)

## OpenAI TTS

In [13]:
import openai, os
from pathlib import Path
openai.api_key = os.getenv("OPENAI_API_KEY")

In [14]:
os.mkdir(f'podcast/{episode}/audio')

In [15]:
voices = ['alloy', 'echo', 'fable', 'onyx', 'nova', 'shimmer']

In [16]:
voices = ['alloy', 'echo', 'fable', 'onyx', 'nova', 'shimmer']
for i in range(len(articles)):
    print(voices[i % 6])

alloy
echo
fable
onyx
nova
shimmer


In [17]:
# Read the text from the file
with open(f'podcast/{episode}/summary/' + article, 'r',encoding='utf-8') as f:
    summary = f.read()

In [18]:
from pathlib import Path
import openai

speech_file_path = Path(f'podcast/{episode}/audio/' + article.replace('.txt', '.mp3'))
response = openai.audio.speech.create(
  model="tts-1",
  voice=voices[0],
  input=summary
)
response.stream_to_file(speech_file_path)

## Elevenlabs

In [35]:
from elevenlabs import generate, play, save, voices, set_api_key
set_api_key(os.getenv('ELEVENLABS_KEY'))

In [37]:
voices_list = list(voices())
voices_list

[Voice(voice_id='21m00Tcm4TlvDq8ikWAM', name='Rachel', category='premade', description=None, labels={'accent': 'american', 'description': 'calm', 'age': 'young', 'gender': 'female', 'use case': 'narration'}, samples=None, settings=VoiceSettings(stability=0.75, similarity_boost=0.75), design=None),
 Voice(voice_id='2EiwWnXFnvU5JabPnv8n', name='Clyde', category='premade', description=None, labels={'accent': 'american', 'description': 'war veteran', 'age': 'middle aged', 'gender': 'male', 'use case': 'video games'}, samples=None, settings=VoiceSettings(stability=0.5, similarity_boost=0.75), design=None),
 Voice(voice_id='AZnzlk1XvdvUeBnXmlld', name='Domi', category='premade', description=None, labels={'accent': 'american', 'description': 'strong', 'age': 'young', 'gender': 'female', 'use case': 'narration'}, samples=None, settings=VoiceSettings(stability=0.5, similarity_boost=0.75), design=None),
 Voice(voice_id='CYw3kZ02Hs0563khs1Fj', name='Dave', category='premade', description=None, la

In [38]:
voice = 'Yann'

# Generate audio for the content
audio = generate(
    text=summary,
    voice=voice,
    model="eleven_multilingual_v2"
)

save(audio,f'podcast/{episode}/audio/' + article.replace('.txt', '.mp3'))

## Polly

In [39]:
import boto3
import os
# Import the necessary libraries

# Set up the AWS credentials
aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
aws_region_name = os.environ.get('AWS_REGION_NAME')

In [40]:
# # Read the text from the file
# with open(f'podcast/{episode}/summary/' + article, 'r') as f:
#     summary = f.read()

# Initialize the Polly client
polly_client = boto3.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_region_name).client('polly')

# Synthesize the speech using the text
response = polly_client.synthesize_speech(
    Text=summary,
    OutputFormat='mp3',
    VoiceId='Lea') # French voices, Lea, Celine, Mathieu, Chantal

# Save the synthesized speech to an MP3 file
with open(f'podcast/{episode}/audio/' + article.replace('.txt', ' - Lea.mp3'), 'wb') as f:
    f.write(response['AudioStream'].read())


In [32]:
# For text longer than 3000 characters, use the following code

# Initialize the Polly client
polly_client = boto3.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_region_name).client('polly')

# Split the text into chunks of less than 3000 characters
chunks = [text[i:i+3000] for i in range(0, len(text), 3000)]

# Initialize an empty byte array to hold the audio stream
audio_stream = bytearray()

for chunk in chunks:
    # Synthesize the speech using the chunk
    response = polly_client.synthesize_speech(
        Text=chunk,
        OutputFormat='mp3',
        VoiceId='Lea') # French voices, Lea, Celine, Mathieu, Chantal

    # Append the audio stream from this chunk to the overall audio stream
    audio_stream.extend(response['AudioStream'].read())

# Save the synthesized speech to an MP3 file
with open(f'podcast/{episode}/audio/' + article.replace('.txt', '.mp3'), 'wb') as f:
    f.write(audio_stream)

## Intro

In [23]:
intro = 'ami_{:03d}.txt'.format(episode_number)

# Define the template
template = '''Conseil d'ami
episode {}

Lundi 13 novembre

Au programme de cet épisode:

'''

# Read the articles from the directory
articles = os.listdir(f'podcast/{episode}/text')

# Initialize an empty string to hold the formatted articles
formatted_articles = template.format(episode_number)

# Loop through the articles and format them
for article in articles:
    formatted_articles += article.replace('txt','') + '\n'

# Write the output to a file
with open(f'podcast/{episode}/' + intro, 'w', encoding='utf-8') as f:
    f.write(formatted_articles)

In [76]:
# %% Intro Elevenlabs
from elevenlabs import generate, save, set_api_key
set_api_key(os.getenv('ELEVENLABS_KEY'))
intro = 'ami_{:03d}.txt'.format(episode_number)
# Read the text from the file
with open(f'podcast/{episode}/' + intro, 'r',encoding='utf-8') as f:
    text = f.read()

voice = 'Yann'

# Generate audio for the content
audio = generate(
    text=text,
    voice=voice,
    model="eleven_multilingual_v2"
)

save(audio,f'podcast/{episode}/intro.mp3')

## Edit

In [42]:
audio_files = [f for f in os.listdir(directory+ '/audio/') if f.endswith('.mp3')]
audio_files

["Chine - les yeux grands fermés de l'Occident.mp3",
 'Comprendre le langage des chiffres.mp3',
 'Electricité - Berlin dégaine 12 milliards pour ses industriels.mp3',
 "Face au manque d'ingénieurs, CentraleSupelec cherche à élargir son recrutement.mp3",
 'Israël accepte des « pauses » quotidiennes à Gaza.mp3']

In [ ]:
from pydub import AudioSegment
import os

def edit(episode_number):
    episode_number = int(episode_number)
    episode = 'episode' + '{:03d}'.format(episode_number)
    # Set the directory for the episode
    directory = 'podcast/' + episode 

    # Get a list of all the audio files in the directory
    audio_files = [f for f in os.listdir(directory+ '/audio/') if f.endswith('.mp3')]

    # Initialize an empty AudioSegment object to hold the concatenated audio
    concatenated_audio = AudioSegment.empty()

    # Add the audio to the concatenated audio
    concatenated_audio += AudioSegment.from_mp3('ami_generic.mp3')

    # Add the intro audio to the concatenated audio
    intro_audio = AudioSegment.from_mp3(directory + '/intro.mp3')
    concatenated_audio += intro_audio

    # Loop through the audio files and append them to the concatenated audio
    for file in audio_files:
        concatenated_audio += AudioSegment.from_mp3('ami_generic.mp3')
        audio = AudioSegment.from_mp3(directory + '/audio/' + file)
        concatenated_audio += audio

    # Export the concatenated audio to a file
    concatenated_audio.export(directory + f'/ami_{episode}.mp3', format='mp3')

edit(episode_number)